In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Wczytanie danych
data = pd.read_csv('penguins.csv', sep=';', decimal=',', encoding='utf-8')

# Zmiana typów kolumn
data = data.astype({'bill_length_mm': 'float', 'bill_depth_mm': 'float', 'flipper_length_mm': 'float', 'body_mass_g': 'float'})

# Lista kolumn numerycznych
numeric_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

# Funkcja usuwająca wartości odstające na podstawie IQR
def filter_outliers(df, cols):
    for column in cols:
        q1, q3 = df[column].quantile([0.25, 0.75])
        iqr = q3 - q1
        lower_bound, upper_bound = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        df.loc[(df[column] < lower_bound) | (df[column] > upper_bound), column] = np.nan
    return df

# Usuwanie wartości odstających
cleaned_data = filter_outliers(data.copy(), numeric_features)

# Uzupełnianie brakujących danych
cleaned_data[numeric_features] = cleaned_data[numeric_features].fillna(cleaned_data[numeric_features].median())
cleaned_data['sex'] = cleaned_data['sex'].fillna(cleaned_data['sex'].mode()[0])

# One-hot encoding dla kolumny 'sex'
encoded_data = pd.get_dummies(cleaned_data, columns=['sex'], drop_first=True)

# Standaryzacja danych numerycznych
scaler = StandardScaler()
encoded_data[numeric_features] = scaler.fit_transform(encoded_data[numeric_features])

# Zmiana nazw kolumn
encoded_data.columns = ['species', 'bill_length', 'bill_depth', 'flipper_length', 'body_mass', 'male_sex']

# Zapis do pliku CSV
encoded_data.to_csv('my-penguins.csv', sep=',', index=False, encoding='utf-8')
